## Module 불러오기 

In [1]:
import os
import cv2
import glob
import json
import timeit
import numpy as np
from random import *
#import tensorflow as tf
from patchify import patchify
import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
from collections import OrderedDict

## 폴더 만들기 

In [22]:
#여기까지는 한 번 확인하기 
base_path = 'C:\\Users\\tndus\\workspace\\DIP\\project'
ori_path='DIP/data/18-40-02-02 (SLA).jpg' #원본 이미지
lbl_img_path='DIP/labelled_data/18-40-02-02 (SLA)_re_image.png' #원본 labelled 이미지

fnr_path='DIP/data/aug/remove_lt.jpg' #글씨 제거 이미지 

#Data Path
DIP_path=os.path.join(base_path,'DIP')
data_path = os.path.join(DIP_path, 'data')
lbl_path = os.path.join(DIP_path, 'labelled_data')

# 원본이미지 어그멘테이션 폴더 
data_aug_path = os.path.join(data_path,'aug')
aug_ver1_path = os.path.join(data_aug_path,'ver_1')
aug_ver2_path = os.path.join(data_aug_path,'ver_2')
aug_patch_path = os.path.join(aug_ver1_path,'patches')
aug2_patch_path = os.path.join(aug_ver2_path,'patches')

# 라벨이미지 어그멘테이션 폴더 
lbl_aug_path = os.path.join(lbl_path,'aug')
lbl_aug_ver1_path = os.path.join(lbl_aug_path,'ver_1')
lbl_aug_ver2_path = os.path.join(lbl_aug_path,'ver_2')
lbl_aug_patch_path = os.path.join(lbl_aug_ver1_path,'patches')
lbl_aug2_patch_path = os.path.join(lbl_aug_ver2_path,'patches')

# json 폴더 
json_path_ver1 = os.path.join(aug_ver1_path,'json')
json_path_ver2= os.path.join(aug_ver2_path,'json')


In [5]:
def mkfolder(folder):

    for j in range(len(folder)):
        if not os.path.exists(folder[j]):
            os.makedirs(folder[j])
  
'''
data_path, train_path는 string 타입의 path
mkfolder안에 입력할 값 = list 타입
'''

'\ndata_path, train_path는 string 타입의 path\nmkfolder안에 입력할 값 = list 타입\n'

In [6]:
fold_list = [DIP_path,data_path,lbl_path,
             data_aug_path,aug_ver1_path,aug_ver2_path,aug_patch_path,aug2_patch_path,
            lbl_aug_path,lbl_aug_ver1_path,lbl_aug_ver2_path,lbl_aug_patch_path,lbl_aug2_patch_path ,
             json_path_ver1, json_path_ver2]

mkfolder(fold_list)

## 밑에 글씨 제거하기

In [7]:
def remove_wr(ori_path,save_path):
    original = cv2.imread(ori_path, cv2.IMREAD_COLOR)
    kernel=np.ones((11,11),np.uint8)
    ori_opened = cv2.morphologyEx(original[1100:1200,1300:1600,:], cv2.MORPH_OPEN, kernel) #글씨 영역 선택 
    original[1100:1200,1300:1600,:] = ori_opened #morphological 연산 중 opening 이용 
    cv2.imwrite(save_path+'/remove_lt.jpg', original) #새로운 폴더에 글씨 제거한 이미지 저장 

In [8]:
remove_wr(ori_path, data_aug_path)

## Labelled Image Red to Black 

In [9]:
def RedtoBlack(lbl_img_path, lbl_path):    
    img=cv2.imread(lbl_img_path)
    height, width, _ = img.shape 
    
    for i in range(height):
        for j in range(width):
            # img[i,j] is the RGB pixel at position (i, j)
            # check if it's [0, 0, 0] and replace with [255, 255, 255] if so
            if (img[i,j])[2] >= 250:
                img[i, j] = [0, 0, 0]
    cv2.imwrite(lbl_path+'/black_lbl_img.jpg',img)

In [10]:
RedtoBlack(lbl_img_path,lbl_path)

## Augmentation Ver.1 
### Rotation & Flip & Translation & Padding

In [ ]:
# 1. rotation
# 2. flip
#aug1_path = os.path.join(aug_path,'aug_ver1') # aug한 이미지 저장할 폴더 

'''
aug_image function:
[arguments]
- path = augmentation적용할 이미지를 불러올 폴더
- deg = float 또는 int type의 rotation시킬 각도(random변수로 앞에서 지정)

[function]
M = cv2.getRotationMatrix2D((x,y),angle,scale)
(x,y) = 회전 중심점
angle = 회전 중심점을 기준으로 회전할 각도
scale = 이미지의 확대 및 축소 비율 (default:1 = 인풋 크기 그대로)
cv2.warpAffine(image, M, (dsize)) = 아핀 변환 함수(cv2.warpAffine)로 회전 변환을 계산
-> M = 계산할 matrix
-> dsize = tuple 타입의 출력 이미지의 너비와 높이.
'''


In [11]:
Extension= '.jpg'

#aug1_path = image 
#aug2_path = lbl_image 

def aug_image(fnr_path, lbl_fnr_path, aug1_path, aug2_path):
    global flip_dic #json 파일 생성을 위해 global 함수로 선언해줌
    image = cv2.imread(fnr_path,cv2.IMREAD_COLOR) #글씨 제거한 사진 input으로 넣어줌
    lbl_image= cv2.imread(lbl_fnr_path,cv2.IMREAD_COLOR) #labelled_data는 원본으로 넣어줌 
    rows,cols,c = image.shape

    
    print("This time, We are going to 'Rotate' and 'Flip' your image.\n")
    print('*------------------------------------------------------------*\n')
    print("First, Input the degree! It wil be the maximum value on your 'countclockwise' rotation range.")
    x = int(input('minimum of degree:'))
    y = int(input('maximum of degree:'))
    print("\n")
    print("Let's move on to 'FLIP'\n")
    print("We'll give you THREE options. Please choose one of them.\n")
    print("OPTION 1: Flip the image vertically, horizontally.")
    print("OPTION 2: Flip the image ONLY vertically.")
    print("OPTION 3: Flip the image ONLY horizontally.\n")
    ans_flip = int(input("What option would you like to choose?(Just enter the option number):"))
    
    # Rotation
    deg = randint(x,y) #회전할 각도를 x, y 사이의 random한 정수로 받음 
    M = cv2.getRotationMatrix2D((cols/2,rows/2), deg, 1) #회전중심, 회전 중심점 기준 회전할 각도, 이미지 확대/축소 비율
    aug = cv2.warpAffine(image, M, (cols, rows))
    aug2 = cv2.warpAffine(lbl_image, M, (cols, rows))
    
    # Flip
    if ans_flip == 1: 
        aug = cv2.flip(aug,1) # horizontal
        aug = cv2.flip(aug,0) # vertical 
        aug2 = cv2.flip(aug2,1) # horizontal
        aug2 = cv2.flip(aug2,0) # vertical 
        cv2.imwrite(aug1_path + "/rot-%s-flipVH"%str(deg)+Extension, aug)
        cv2.imwrite(aug2_path + "/rot-%s-flipVH"%str(deg)+Extension, aug2)
        v, h ='yes','yes'
    elif ans_flip == 2: #vertical flip only
        aug = cv2.flip(aug,0)
        aug2 = cv2.flip(aug2,0) 
        cv2.imwrite(aug1_path + "/rot-%s-flipV"%str(deg)+Extension, aug)
        cv2.imwrite(aug2_path + "/rot-%s-flipV"%str(deg)+Extension, aug2)
        v, h = 'yes','no'
    elif ans_flip == 3: #horizontal flip only
        aug = cv2.flip(aug,1)
        aug2 = cv2.flip(aug2,1)
        cv2.imwrite(aug1_path + "/rot-%s-flipH"%str(deg)+Extension, aug)
        cv2.imwrite(aug2_path + "/rot-%s-flipH"%str(deg)+Extension, aug2)
        v, h = 'no','yes'

    else:
        print('[WARNING : TRY AGAIN]')
    
    flip_dic = {'rotation range (degree)':[x,y],'rotation degree':deg,'vertical flip':v,'horizental flip': h}
    return flip_dic

In [12]:
lbl_img_path=lbl_path+'/black_lbl_img.jpg'
aug_image(fnr_path, lbl_img_path, aug_ver1_path, lbl_aug_ver1_path)

This time, We are going to 'Rotate' and 'Flip' your image.

*------------------------------------------------------------*

First, Input the degree! It wil be the maximum value on your 'countclockwise' rotation range.
minimum of degree:-20
maximum of degree:10


Let's move on to 'FLIP'

We'll give you THREE options. Please choose one of them.

OPTION 1: Flip the image vertically, horizontally.
OPTION 2: Flip the image ONLY vertically.
OPTION 3: Flip the image ONLY horizontally.

What option would you like to choose?(Just enter the option number):1


{'rotation range (degree)': [-20, 10],
 'rotation degree': -7,
 'vertical flip': 'yes',
 'horizental flip': 'yes'}

### PadToFixedSize (Padding + Translation)

In [13]:
#이후에 128 x 128 사이즈로 patch를 잘라주기 때문에 잘라주면서 남는 공간이 생기지 않도록 128의 배수를 가진 사이즈로 padding 시켜줌 

def padding(img,lbl_img, aug1_path, aug2_path, patch_size):
    global a, b
    x=img.shape[1] #사진의 width = column
    y=img.shape[0] #사진의 height = row

    maa=patch_size*(max(x,y)//patch_size+2) #128의 배수로 만들기 위해 처리해줌 
    mii=patch_size*(max(x,y)//patch_size+1)

    if min == x:
        a = maa
        b = mii
        print('width')
    else:
        a = mii
        b = maa
        print('height')

    a = randrange(mii, maa, patch_size) #mii, maa 사이에서 step=128 (128의 배수)인 random한 정수
    b = randrange(mii, maa, patch_size) 

    aug=iaa.PadToFixedSize(width=b, height=a, position='left-top') #padding을 수행하여 이미지 사이즈로 만들어줌 + translation
    pad=aug(image=img)
    pad_label=aug(image=lbl_img)

    cv2.imwrite(aug1_path + '/af_pad.jpg',pad)
    cv2.imwrite(aug2_path +'/af_pad.jpg',pad_label)

In [14]:
patch_size=128   

img_path=glob.glob(aug_ver1_path+'/rot*.jpg') #glob 함수를 이용하여 폴더 내 rot로 시작하는 이름을 가진 flip&rotate마친 후 파일 불러들임 
img=cv2.imread(img_path[0])

lbl_rot_path = glob.glob(lbl_aug_ver1_path+'/rot*.jpg')
lbl_img= cv2.imread(lbl_rot_path[0]) 

padding(img,lbl_img, aug_ver1_path, lbl_aug_ver1_path, patch_size)

height


## Patch 자르고 저장하기 

In [15]:
#patch_size=128

def save_patches(pad_path,save_path,patch_size):
    global p_num
    img = cv2.imread(pad_path) #padding 후의 이미지 input으로 넣어주기
    # one image -> patches
    patches = patchify(img,(patch_size,patch_size,3), step=patch_size)
 
    #patach 저장시키는 코드
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            single_patch = patches[i,j,0,:,:,:]
            cv2.imwrite(save_path+'/img_%s_%s.jpg'%(str(i),str(j)),single_patch)
            p_num = (i+1)*(j+1) #patch의 개수 반환해줌 
    return p_num

In [16]:
pad_path = aug_ver1_path+'/af_pad.jpg'
lbl_pad_path=lbl_aug_ver1_path+'/af_pad.jpg'

save_patches(pad_path,aug_patch_path,patch_size)
save_patches(lbl_pad_path,lbl_aug_patch_path,patch_size)

169

In [17]:
def mkJson(JSON):
    file_data = OrderedDict()

    file_data['Translation(padding)'] = {'paddingBox_height':JSON[0],'paddingBox_width':JSON[1]}
    file_data['Rotation_counterClockwise'] = {'range(dgree)':JSON[2],'degree':JSON[3]}
    file_data['Flip'] = {'vertical':JSON[4], 'horizontal':JSON[5]}
    file_data['number of patches'] = JSON[6]
    file_data['size of patches'] = JSON[7]
    
    file_data['contrast alpha'] = JSON[8]
    file_data['shear'] = JSON[9]
    
    print(json.dumps(file_data,ensure_ascii=False,indent='\t'))
    if JSON[8] == None:
        with open(json_path_ver1 + '/descriptor.json','w',encoding='utf-8') as make_file:
            json.dump(file_data,make_file,ensure_ascii=False,indent='\t')
    else:
        with open(json_path_ver2 + '/descriptor.json','w',encoding='utf-8') as make_file:
            json.dump(file_data,make_file,ensure_ascii=False,indent='\t')
            
# JSON = [a,b,flip_dict['rotation range (degree)'],flip_dict['rotation degree'],flip_dict['vertical flip'],flip_dict['horizental flip'],p_num]
# mkJson(JSON

In [18]:
JSON = [a,b,
        flip_dic['rotation range (degree)'],
        flip_dic['rotation degree'],
        flip_dic['vertical flip'],flip_dic['horizental flip'],
        p_num,patch_size,None,None]

mkJson(JSON)

{
	"Translation(padding)": {
		"paddingBox_height": 1664,
		"paddingBox_width": 1664
	},
	"Rotation_counterClockwise": {
		"range(dgree)": [
			-20,
			10
		],
		"degree": -7
	},
	"Flip": {
		"vertical": "yes",
		"horizontal": "yes"
	},
	"number of patches": 169,
	"size of patches": 128,
	"contrast alpha": null,
	"shear": null
}


## Augmentation Ver.2 
### Shear + GrayScale + Contrast + Flip + Rotation + Translation + Padding

In [24]:
def shear(fnr_path, lbl_img_path):
    global shear
    src = cv2.imread(fnr_path) 
    lbl_src=cv2.imread(lbl_path+'/black_lbl_img.jpg')

    shear=uniform(0,2)
    aff = np.array([[1, 0.5, 0], [0, 1, 0]], dtype=np.float32)

    h, w = src.shape[:2] 
    dst=cv2.warpAffine(src, aff, (w + int(h * shear), h)) #x축을  y축 대비 shear의 이븅로 기울인 효과 
    lbl_dst=cv2.warpAffine(lbl_src, aff, (w + int(h * shear), h))
    cv2.imwrite(aug_ver2_path+'/shear.jpg',dst)
    cv2.imwrite(lbl_aug_ver2_path+'/shear.jpg',lbl_dst)
    return shear

#dst의 크기는 affine 변환 행렬에서 x축 방향으로 늘어난 만큼 더 더해주어야 합니다.
# affine 변환 행렬에서 x축의 사이즈가 늘어난 크기는 y축 사이즈의 shear배 만큼 늘어나게 되므로 (h*shear)를 w에 더해줍니다.

In [25]:
lbl_img_path=lbl_path+'/black_lbl_img.jpg'
shear(fnr_path,lbl_img_path)

0.4515845132460201

In [36]:
Extension='.jpg'
def contrast(min_val, max_val):
    global alpha
    #gray = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    image=cv2.imread(aug_ver2_path+'/shear.jpg',cv2.IMREAD_COLOR)
    lbl_image=cv2.imread(lbl_aug_ver2_path+'/shear.jpg',cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #이미지 gray scale로 변환시켜줌 
    #gray_lbl = cv2.cvtColor(lbl_image, cv2.COLOR_BGR2GRAY)
    alpha = int(uniform(min_val, max_val))*0.4 #Contrast control,사용자가 입력하는 범위 내에서 랜덤으로 alpha값 적용
    adjusted = cv2.convertScaleAbs(gray, alpha=alpha) 
    lbl_adjusted = cv2.convertScaleAbs(lbl_image, alpha=alpha)
    #lbl_adjusted = cv2.convertScaleAbs(gray_lbl, alpha=alpha)
    cv2.imwrite(aug_ver2_path + "/contrast-%s"%str(alpha)+Extension, adjusted)
    cv2.imwrite(lbl_aug_ver2_path + "/contrast-%s"%str(alpha)+Extension, lbl_adjusted)
    #return alpha

In [37]:
contrast(1, 5)

In [28]:
fnr_path=glob.glob(aug_ver2_path+'/contrast*.jpg') #contrast 진행 후 이미지를 glob함수로 불러들여서 flip & rotate 진행 
fnr_path=fnr_path[0]

lbl_fnr_path=glob.glob(lbl_aug_ver2_path+'/contrast*.jpg')
lbl_fnr_path=lbl_fnr_path[0]


aug_image(fnr_path, lbl_fnr_path, aug_ver2_path, lbl_aug_ver2_path)

This time, We are going to 'Rotate' and 'Flip' your image.

*------------------------------------------------------------*

First, Input the degree! It wil be the maximum value on your 'countclockwise' rotation range.
minimum of degree:-20
maximum of degree:30


Let's move on to 'FLIP'

We'll give you THREE options. Please choose one of them.

OPTION 1: Flip the image vertically, horizontally.
OPTION 2: Flip the image ONLY vertically.
OPTION 3: Flip the image ONLY horizontally.

What option would you like to choose?(Just enter the option number):3


{'rotation range (degree)': [-20, 30],
 'rotation degree': 7,
 'vertical flip': 'no',
 'horizental flip': 'yes'}

In [29]:
img_path=glob.glob(aug_ver2_path+'/rot*.jpg')
img=cv2.imread(img_path[0])

lbl_path = glob.glob(lbl_aug_ver2_path+'/rot*.jpg')
lbl_img= cv2.imread(lbl_path[0])

padding(img,lbl_img, aug_ver2_path, lbl_aug_ver2_path, patch_size)

height


In [33]:
#patch_size=128

def save_patches_ver2(pad_path,save_path,patch_size):
    global p_num
    img = cv2.imread(pad_path) #padding 후의 이미지 input으로 넣어주기
    # one image -> patches
    patches = patchify(img,(patch_size,patch_size,3), step=patch_size)
 
    #patach 저장시키는 코드
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            single_patch = patches[i,j,0,:,:,:]
            cv2.imwrite(save_path+'/v2_img_%s_%s.jpg'%(str(i),str(j)),single_patch)
            p_num = (i+1)*(j+1) #patch의 개수 반환해줌 
    return p_num

In [35]:
pad_path = aug_ver2_path+'/af_pad.jpg'
lbl_pad_path=lbl_aug_ver2_path+'/af_pad.jpg'

save_patches_ver2(pad_path,aug2_patch_path,patch_size)
save_patches_ver2(lbl_pad_path,lbl_aug2_patch_path,patch_size)

289

## JSON

In [31]:
JSON = [a,b,
        flip_dic['rotation range (degree)'],
        flip_dic['rotation degree'],
        flip_dic['vertical flip'],flip_dic['horizental flip'],
        p_num, patch_size, alpha, shear]

mkJson(JSON)

{
	"Translation(padding)": {
		"paddingBox_height": 2176,
		"paddingBox_width": 2176
	},
	"Rotation_counterClockwise": {
		"range(dgree)": [
			-20,
			30
		],
		"degree": 7
	},
	"Flip": {
		"vertical": "no",
		"horizontal": "yes"
	},
	"number of patches": 289,
	"size of patches": 128,
	"contrast alpha": 0.8,
	"shear": 0.4515845132460201
}
